In [19]:
import time

with open("./../data/test_performance_input.txt", "r") as f:
    test_input = [input_str.strip() for input_str in f.read().split("###")]
    test_input = [text for text in test_input if text != ""]

len(test_input)

21

In [23]:
import httpx
import json

test_results = []

def run_test(input: str, index: int = -1):
    start = time.time()
    result = httpx.post("http://localhost:8000/processText", json={
        "text": input
    }, timeout=None)
    end = time.time()
    duration = end - start
    test_results.append({"duration": duration, "input": input, "result": result.json()})
    print(f"\033[4mTest result {index}:\033[0m")
    print(f"\033[1mDuration:\033[0m {duration}s")
    print(f"\033[1mInput:\033[0m \"{input}\"")
    for term in result.json()["terms"]:
        print(f"- {term['text'] if term['normalization'] == None else term['normalization']}")
        for definition in term["definitions"]:
            prefix = ""
            if definition["partial"]:
                prefix += "⚙️"
            if not definition["verified"]:
                prefix += "✨"
            else:
                prefix += "☑️"
            print(f"\t- {prefix} {definition['text']}")
    print()

for index, text in enumerate(test_input, start=1):
    try:
        run_test(text, index=index)
    except:
        print(f"Test failed: {text}")

with open("./../data/test_performance_cvalue_output.json", "w") as f:
    f.write(json.dumps(test_results))

Test result 1:
Duration: 0.7679691314697266s
Input: "Hier ist der 12772, gerade angekommen in Gleis 31.
Ich habe jetzt abgehängt und Hemmschuhe unter die ersten beiden Wagen gelegt und eine Handbremse ist auch fest."
- ersten beiden wagen
- 31.
- hemmschuhe
- ersten
- wagen
- handbremse

Test result 2:
Duration: 0.7423641681671143s
Input: "Die Effizienz von Transformatoren kann durch Wirbelstromverluste und magnetische Hysterese beeinträchtigt werden. Was bei der Dimensionierung zu berücksichtigen ist."
- effizienz von transformatoren
- magnetische hysterese
- effizienz
- transformatoren
- wirbelstromverluste
- magnetische
- hysterese
- dimensionierung

Test result 3:
Duration: 0.7041819095611572s
Input: "Feuerlöscher sind tragbare Geräte, die damals entwickelt wurden, um kleine Brände in der Anfangsphase zu löschen."
- tragbare geräte
- kleine brände
- feuerlöscher
- tragbare
- geräte
- kleine
- brände
- anfangsphase

Test result 4:
Duration: 0.7359318733215332s
Input: "Die Verwendung

## Evaluate C-Value gold labels

In [18]:
with open("./../data/test_performance_expected.txt", "r") as f:
    expected = f.read().split("###")
    expected = [text.strip().split("\t") for text in expected]


for test_result, expected in zip(test_results, expected):
    terms = test_result["result"]["terms"]
    actual = [term["text"] if term["normalization"] is None else term["normalization"] for term in terms]


6/6
8/3
8/3
15/3
10/4
11/7
11/5
1/1
12/4
15/3
7/2
12/5
1/1
10/6
9/5
2/2
1/4
4/1
1/1
11/2
6/5


In [13]:
import json

with open("./../data/test_performance_output.json", "r") as f:
    test_results = json.loads(f.read())

with open("./../data/test_performance_expected.txt", "w") as f:
    f.write("\n\n###\n\n".join("\t".join(sample["expected"]) for sample in test_results))